# Modeling and Data Analysis

This is an example "walk through" of one of the [labs](https://github.com/sspickle/instrumentation-projects/blob/master/proj1/Arduino%20as%20a%20Laboratory%20Interface.ipynb) we do in our sophmore [electronics course](https://github.com/sspickle/instrumentation-projects). The course has two primary learning outcomes:

Students learn to apply:

1. a variety of computational modeling and statistical techniques centering on experimental data analysis AND
2. physics concepts from intro electricity and magnetism in a laboratory setting through the use of electronics and microcontrollers

A typical lab would involve

1. building a circuit meant to accomplish some experimental goal, 
2. writing a microcontroller program to run the experiment and collect data, 
3. apply a statistical/analysis approach to make inferences from the collected data

## A generative model

One of the important ideas we study is the notion of a generative model. This is a mathematical model of the system that includes some measure of randomness or noise, usually represented as a normal distribution superposed with an exact deterministic model. A very simple example from the intro physics course is Ohm's Law. If we measure the voltage accross a resistor in the lab we'd expect it to depend on the current in the expected way:

$$ V = I R $$

A generative model might look like this:

$$ V = I R + \delta V$$

where $\delta V$ is a random contribution from noise. We can model this in python using the numpy, scipy, and matplotlib libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pandas as pd

In [ ]:
#
# first let's create an array of current values:
#

Rg = 1000.0  # 1 k Ohm
sig = 0.005  # 5 mV

I = np.linspace(0, 0.010, 11)
I

In [ ]:
V = I * Rg + sig*np.random.randn(len(I))
V

In [ ]:
plt.plot(I, V, 'o')
plt.xlabel('Current (A)')
plt.ylabel('Voltage (V)')
plt.title("V-I curve for a 1 kOhm resistor")
plt.grid()

In [ ]:
#
# Let's define the model function
#

def OhmsLaw(I, R, offset):
    return I * R + offset

# curve_fit takes 1) function, 2) x, 3) y, 4) initial guess (p0, optional) 5) uncertainties (sigma, optional)
(R, offset), cov = curve_fit(OhmsLaw, I, V, p0=[Rg,0], sigma=np.ones(len(I))*sig) 

dR,doffset = np.sqrt(np.diag(cov))          # returns tuple of parameters and covariance matrix

print("R = %f +/- %f Ω" % (R, dR))
print("offset = %f +/- %f V" % (offset, doffset))

In [ ]:
Vth = OhmsLaw(I, R, offset)
plt.plot(I, V, 'o', label='data')
plt.plot(I, Vth, '-', label='fit')
plt.xlabel('Current (A)')
plt.ylabel('Voltage (V)')
plt.title("V-I curve for a 1 kOhm resistor")
plt.grid()
plt.legend()

chisq = np.sum(((V - Vth)/sig)**2)/(len(I)-2)
print("chisq = %f" % chisq)

The value of the generative model is the ability to validate the data analysis and inference approach. If you can generate data with the same statistical properties as the data you *expect* to measure, and the data analysis approach produces inferences that are consistent with the generative model, it lends credibility to that process.




## The Schockley Diode Equation

The first bit of actual "electronics" we encounter in the course is the PN junction, and the form of a light emitting diode (LED). These are non-linear devices that very nearly obey a theoretical relationship:

$$ I = I_o (e^{\frac{q V_d}{\eta k_b T}}-1) $$

Where $V_d$ is the voltage drop across the PN junction, $\eta$ is the "emission coefficient", $q$ is the electron charge, $k_b$ is the Boltzmann constant and $T$ is the absolute temperature. Notice that when the LED is drawing appreciable current we're in a domain where $q V_d/\eta$ is much greater than $k_b T$, so the $-1$ in parenthesis is negligable compared to the exponential. So our simplifed ideal diode model becomes:

$$ I \approx I_o e^{\frac{ q V_d}{\eta k_b T}} $$

Now, solve this for $V_d$ as a function of $I$:

$$ V_d = \frac{\eta k_b T}{q} \ln(I/I_o)$$

You can think of this as the "Ohm's Law" of an LED. We can simplify this by defining $A$ and $B$ like so:

$$ A=\frac{\eta k_b T}{q}$$

$$B = -A \ln(I_o) $$

which gives:

$$ V_d = A \ln(I) + B $$

If we graph $V_d$ vs $I$ we should get a straight line! Nice.



# A Generative Model

Let's build a generative model for this theory. Usually $\eta$ is not too far from unity, and $I_0$ is extremely small. For the purposes of having a generative model, let's pick $\eta \approx 1.5$ and $I_0 \approx 1\times 10^{-21}$

In [ ]:
Igen = np.linspace(0, 0.010, 100)[1:] # exclude I=0
kbToQ = 0.0259 # V
eta_gen = 1.5
I0_gen = 1e-21
Agen = eta_gen * kbToQ
Bgen = -Agen*np.log(I0_gen)
sig = 0.005 # ~2 x minimum resolution of the ADC
Vgen = Agen * np.log(Igen) + Bgen + sig*np.random.randn(len(Igen))

plt.plot(np.log(Igen), Vgen, 'o', label="gen model")
plt.xlabel('log(I) (A)')
plt.ylabel('Diode Voltage (V)')
plt.title("Generative Model: V-I curve for a diode")
plt.grid()

So, that doesn't look completely crazy. Let's fit that and see how it goes:

In [ ]:
def linear_model(lnI, A, B):
    return A*lnI + B

lnI = np.log(Igen)

(A, B), cov = curve_fit(linear_model, lnI, Vgen, sigma=np.ones(len(Vgen))*sig)
dA,dB = np.sqrt(np.diag(cov))          # returns tuple of parameters and covariance matrix

plt.plot(lnI,Vgen,'b.', label="data")

Vth = linear_model(lnI, A, B)
plt.plot(lnI, Vth, 'r-',label="fit")
plt.title("LED V-I curve, log scale")
plt.ylabel("Diode voltage (V)")
plt.xlabel("ln(I) (log of amperes)")
plt.grid()
plt.legend()

print("A = %f +/- %f" % (A, dA))
print("B = %f +/- %f V" % (B, dB))

chiSq = np.sum(((Vgen - linear_model(lnI, A, B))/sig)**2)
print("chiSq = %f" % (chiSq/(len(Vgen)-2)))


In [ ]:
eta = A/kbToQ
print(eta)

lnI0 = -B/A
I0 = np.exp(lnI0)
print(I0)

In [ ]:
Amc = A + dA*np.random.normal(size=10000)
Bmc = B + dB*np.random.normal(size=10000)
etaMC = Amc/kbToQ
I0MC = np.exp(-Bmc/Amc)
plt.hist(etaMC, bins=20)

In [ ]:
plt.hist(I0MC, bins=20)

# Experiment

The basic experiment involves driving an LED with varying amounts of current and measuring the voltage drop to witness the non-linear behavior of the device.

## First Pass

We do a "first pass" of this experiment early in the course (first lab!) as an introduction to microcontrollers (e.g., Arduino Uno) and collecting data. The circuit the students build looks like this. Notice that since the Uno has no D/A converter we have to drive the LED with a PWM pin which requires substantial filtering.

![circuit](https://raw.githubusercontent.com/sspickle/SJS-PICUP-2023/129debb2a33d2436b457fe92ef5bacbcf79a00db/proj1.png)

## Improvement

Later in the course, after we've learned a bit about transistors, we work with this version of the circuit:

![circuit2](https://raw.githubusercontent.com/sspickle/SJS-PICUP-2023/129debb2a33d2436b457fe92ef5bacbcf79a00db/proj1a.png)

This is a much better circuit because it makes the filtering much easier with low currents (and a much smaller capacitor). It also highlights the advantage of driving a load from the Arduino using a BJT with a ~200x current gain. As far as data analysis is concerned there's very little difference between the two circuits, but the data we get is much cleaner with this version.

The Arduino code typically looks like this:

![code](https://raw.githubusercontent.com/sspickle/SJS-PICUP-2023/129debb2a33d2436b457fe92ef5bacbcf79a00db/micro-code.png)

Let's go ahead and load that data!


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/sspickle/SJS-PICUP-2023/main/LED_data.csv')
df.head()

In [ ]:
df.plot()
plt.grid()
plt.title("LED raw data")
plt.xlabel("sample number")
plt.ylabel("Values (raw)")

In [ ]:
vc = (df.A1*5/1023).values
ve = (df.A0*5/1023).values
plt.plot(df.drive, vc, label="Vc")
plt.plot(df.drive, ve, label="Ve")
plt.grid()
plt.title("collector and emitter voltages during run")
plt.xlabel("loop iteration number (drive)")
plt.ylabel("voltage")
plt.legend()

In [ ]:
Re = 220
Id = ve/Re
Vd = 5-vc
valid = (Id > 0) # boolean array, true when Id > 0
plt.plot(Vd[valid],Id[valid]*1000,'b.')
plt.grid()
plt.title("I-V curve of LED")
plt.xlabel("Diode voltage (V)")
plt.ylabel("Diode current (mA)")

In [ ]:
lnI = np.log(Id[valid])
plt.plot(lnI,Vd[valid],'b.')
plt.title("LED V-I curve, log scale")
plt.ylabel("Diode voltage (V)")
plt.xlabel("ln(I) (log of amperes)")
plt.grid()

In [ ]:
def linear_model(lnI, A, B):
    return A*lnI + B

lnI = np.log(Id[valid])

A0 = (Vd[valid][-1] - Vd[valid][0])/(lnI[-1] - lnI[0])
B0 = Vd[valid][0] - A0*lnI[0]

(A, B), cov = curve_fit(linear_model, lnI, Vd[valid], p0=[A0,B0], sigma=np.ones(len(Vd[valid]))*sig)
dA,dB = np.sqrt(np.diag(cov))          # returns tuple of parameters and covariance matrix

plt.plot(lnI,Vd[valid],'b.', label="data")

Vth = linear_model(lnI, A, B)
plt.plot(lnI, Vth, 'r-',label="fit")
plt.title("LED V-I curve, log scale")
plt.ylabel("Diode voltage (V)")
plt.xlabel("ln(I) (log of amperes)")
plt.grid()
plt.legend()

print("A = %f +/- %f" % (A, dA))
print("B = %f +/- %f V" % (B, dB))

chiSq = np.sum(((Vd[valid] - linear_model(lnI, A, B))/sig)**2)
print("chiSq = %f" % (chiSq/(len(Vd[valid])-2)))


In [ ]:
def linear_model_plusRint(I, A, B, Rint):
    return A*np.log(I) + B + Rint*I

lnI = np.log(Id[valid])
A0 = (Vd[valid][-1] - Vd[valid][0])/(lnI[-1] - lnI[0])
B0 = Vd[valid][0] - A0*lnI[0]
R0 = 0

(A, B, Rint), cov = curve_fit(linear_model_plusRint, Id[valid], Vd[valid], p0=[A0,B0,R0], sigma=np.ones(len(Vd[valid]))*sig)
dA,dB,dR = np.sqrt(np.diag(cov))          # returns tuple of parameters and covariance matrix

plt.plot(lnI,Vd[valid],'b.', label="data")

Vth = linear_model_plusRint(Id[valid], A, B,Rint)
plt.plot(lnI, Vth, 'r-',label="fit")
plt.title("Non linear model fit to LED V-I curve")
plt.ylabel("Diode voltage (V)")
plt.xlabel("ln(I) (log of amperes)")
plt.grid()
plt.legend()

print("A = %f +/- %f" % (A, dA))
print("B = %f +/- %f" % (B, dB))
print("Rint = %f +/- %f" % (Rint, dR))

chiSq = np.sum(((Vd[valid] - linear_model_plusRint(Id[valid], A, B, Rint))/sig)**2)
print("chiSq = %f" % (chiSq/(len(Vd[valid])-3)))


In [ ]:
eta = A/kbToQ
print(eta)

lnI0 = -B/A
I0 = np.exp(lnI0)
print(I0)

In [ ]:
Amc = A + dA*np.random.normal(size=10000)
Bmc = B + dB*np.random.normal(size=10000)
etaMC = Amc/kbToQ
I0MC = np.exp(-Bmc/Amc)
plt.hist(etaMC, bins=20)

In [ ]:
plt.hist(I0MC, bins=20)